In [86]:
%run header.py
%matplotlib inline

from utils import sample_from_interactions
from utils import from_interactions_to_coo
from utils import combine_with_identity

from sklearn.model_selection import train_test_split as TTS

In [2]:
exp_path = '/nmnt/x04-hdd/boris_temp/SGIMC_IMC/movielens/'

In [3]:
PROBLEM = "classification" if False else "regression"

step_fn = step_qaadmm

if PROBLEM == "classification":
    QAObjectiveLoss = QAObjectiveLogLoss
else:
    QAObjectiveLoss = QAObjectiveL2Loss  # QAObjectiveHuberLoss

## Data loading

In [87]:
interactions = np.load(exp_path + 'I.npy')
X = np.load(exp_path + 'X.npy')
Y = np.load(exp_path + 'Y.npy')

n = len(np.unique(interactions[0])) # users 
m = len(np.unique(interactions[1])) # items


X_comb = combine_with_identity(X)
Y_comb = combine_with_identity(Y)

In [ ]:
def divide_train_test(I, X, Y, shape, q, seed=42):
    # TODO: add q1 and q2 instead q
    n, m = shape
    user_ids = np.array([i for i in range(n)])
    item_ids = np.array([i for i in range(m)])
    
    train_user_ids, test_user_ids = TTS(user_ids, train_size=q, random_state=seed, shuffle=False)
    train_item_ids, test_item_ids = TTS(item_ids, train_size=q, random_state=seed, shuffle=False)
    
    train_train, train_test, test_train, test_test = [], [], [], []
    for elem in tqdm(I.T):
        if elem[0] in train_user_ids and elem[1] in train_item_ids:
            train_train.append(elem)
        elif elem[0] in train_user_ids and elem[1] in test_item_ids:
            train_test.append(elem)
        elif elem[0] in test_user_ids and elem[1] in train_item_ids:
            test_train.append(elem)
        else:
            test_test.append(elem)
            

In [97]:
interactions.T

array([[ 196,  242,    3],
       [ 186,  302,    3],
       [  22,  377,    1],
       ...,
       [ 276, 1090,    1],
       [  13,  225,    2],
       [  12,  203,    3]])

In [5]:
R_full = from_interactions_to_coo(interactions)
full_mask = R_full.toarray() > 0

## n_elem = 0.8

In [14]:
C_ridge = 1e-4
C_group = 3e-4
C_lasso = 10 * C_group
C = (C_lasso, C_group, C_ridge)

eta = 5e0

step_kwargs = {
    "C": C,                 # the regularizr constants (C_lasso, C_group, C_ridge)
    "eta": eta,             # the eta of the ADMM (larger - faster but more unstable)
    "rtol": 1e-4,           # the relative tolerance for stopping the ADMM
    "atol": 1e-7,           # the absolute tolerance
    "method": "cg",         # the method to use in Sub_0
    "n_iterations": 2,      # the number of iterations of the inner ADMM
}

n_iter = 100
K = 10

n_elem = 0.8

In [15]:
interaction_train, interaction_test = sample_from_interactions(interactions, n_elem)

R_train = from_interactions_to_coo(interaction_train, R_shape=R_full.shape).tocsr()
train_mask = R_train.toarray() > 0
test_mask = from_interactions_to_coo(interaction_test, R_shape=R_full.shape).toarray() > 0

/nmnt/media/home/boris/env3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [16]:
problem = IMCProblem(QAObjectiveLoss, X_comb, Y_comb, R_train, n_threads=8)

W_0 = random_state.normal(size=(X_comb.shape[1], K))
H_0 = random_state.normal(size=(Y_comb.shape[1], K))

W, H = W_0.copy(), H_0.copy()

W, H = imc_descent(problem, W, H,
                   step_fn,                  # the inner optimization
                   step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                   n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                   n_init_iterations=0,
                   return_history=True,      # Record the evolution of the matrices (W, H)
                   rtol=1e-5,                # relative stopping tolerance for the outer iterations
                   atol=1e-7,                # absolute tolerance
                   verbose=True,             # show the progress bar
                   check_product=True,       # use the product W H' for stopping
                   )

0it [00:00, ?it/s]
100%|██████████| 100/100 [00:05<00:00, 17.48it/s]


In [17]:
R_hat = get_prediction(X_comb, W, H, Y_comb)

print(relative_loss(R_full.toarray(), R_hat, mask=test_mask))
print(relative_loss(R_full.toarray(), R_hat, mask=train_mask))

0.10411037852965176
0.09156078318939746


## n_elem = 0.7

In [18]:
C_ridge = 1e-4
C_group = 3e-4
C_lasso = 10 * C_group
C = (C_lasso, C_group, C_ridge)

eta = 5e0

step_kwargs = {
    "C": C,                 # the regularizr constants (C_lasso, C_group, C_ridge)
    "eta": eta,             # the eta of the ADMM (larger - faster but more unstable)
    "rtol": 1e-4,           # the relative tolerance for stopping the ADMM
    "atol": 1e-7,           # the absolute tolerance
    "method": "cg",         # the method to use in Sub_0
    "n_iterations": 2,      # the number of iterations of the inner ADMM
}

n_iter = 100
K = 10

n_elem = 0.7

In [19]:
interaction_train, interaction_test = sample_from_interactions(interactions, n_elem)

R_train = from_interactions_to_coo(interaction_train, R_shape=R_full.shape).tocsr()
train_mask = R_train.toarray() > 0
test_mask = from_interactions_to_coo(interaction_test, R_shape=R_full.shape).toarray() > 0

/nmnt/media/home/boris/env3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [20]:
problem = IMCProblem(QAObjectiveLoss, X_comb, Y_comb, R_train, n_threads=8)

W_0 = random_state.normal(size=(X_comb.shape[1], K))
H_0 = random_state.normal(size=(Y_comb.shape[1], K))

W, H = W_0.copy(), H_0.copy()

W, H = imc_descent(problem, W, H,
                   step_fn,                  # the inner optimization
                   step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                   n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                   n_init_iterations=0,
                   return_history=True,      # Record the evolution of the matrices (W, H)
                   rtol=1e-5,                # relative stopping tolerance for the outer iterations
                   atol=1e-7,                # absolute tolerance
                   verbose=True,             # show the progress bar
                   check_product=True,       # use the product W H' for stopping
                   )

0it [00:00, ?it/s]
100%|██████████| 100/100 [00:07<00:00, 12.74it/s]


In [21]:
R_hat = get_prediction(X_comb, W, H, Y_comb)

print(relative_loss(R_full.toarray(), R_hat, mask=test_mask))
print(relative_loss(R_full.toarray(), R_hat, mask=train_mask))

0.10402310376532233
0.07513154157350502


## n_elem = 0.6

In [50]:
C_ridge = 1e-5
C_group = 2e-4
C_lasso = 10 * C_group
C = (C_lasso, C_group, C_ridge)

eta = 7e0

step_kwargs = {
    "C": C,                 # the regularizr constants (C_lasso, C_group, C_ridge)
    "eta": eta,             # the eta of the ADMM (larger - faster but more unstable)
    "rtol": 1e-4,           # the relative tolerance for stopping the ADMM
    "atol": 1e-7,           # the absolute tolerance
    "method": "cg",         # the method to use in Sub_0
    "n_iterations": 2,      # the number of iterations of the inner ADMM
}

n_iter = 100
K = 10

n_elem = 0.6

In [51]:
interaction_train, interaction_test = sample_from_interactions(interactions, n_elem)

R_train = from_interactions_to_coo(interaction_train, R_shape=R_full.shape).tocsr()
train_mask = R_train.toarray() > 0
test_mask = from_interactions_to_coo(interaction_test, R_shape=R_full.shape).toarray() > 0

/nmnt/media/home/boris/env3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [52]:
problem = IMCProblem(QAObjectiveLoss, X_comb, Y_comb, R_train, n_threads=8)

W_0 = random_state.normal(size=(X_comb.shape[1], K))
H_0 = random_state.normal(size=(Y_comb.shape[1], K))

W, H = W_0.copy(), H_0.copy()

W, H = imc_descent(problem, W, H,
                   step_fn,                  # the inner optimization
                   step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                   n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                   n_init_iterations=0,
                   return_history=True,      # Record the evolution of the matrices (W, H)
                   rtol=1e-5,                # relative stopping tolerance for the outer iterations
                   atol=1e-7,                # absolute tolerance
                   verbose=True,             # show the progress bar
                   check_product=True,       # use the product W H' for stopping
                   )

0it [00:00, ?it/s]
100%|██████████| 100/100 [00:07<00:00, 12.94it/s]


In [53]:
R_hat = get_prediction(X_comb, W, H, Y_comb)

print(relative_loss(R_full.toarray(), R_hat, mask=test_mask))
print(relative_loss(R_full.toarray(), R_hat, mask=train_mask))

0.15825099880872626
0.0750629258120321


## n_elem = 0.5

In [82]:
C_ridge = 1e-4
C_group = 3e-4
C_lasso = 10 * C_group
C = (C_lasso, C_group, C_ridge)

eta = 5e0

step_kwargs = {
    "C": C,                 # the regularizr constants (C_lasso, C_group, C_ridge)
    "eta": eta,             # the eta of the ADMM (larger - faster but more unstable)
    "rtol": 1e-4,           # the relative tolerance for stopping the ADMM
    "atol": 1e-7,           # the absolute tolerance
    "method": "cg",         # the method to use in Sub_0
    "n_iterations": 2,      # the number of iterations of the inner ADMM
}

n_iter = 100
K = 10

n_elem = 0.5

In [83]:
interaction_train, interaction_test = sample_from_interactions(interactions, n_elem)

R_train = from_interactions_to_coo(interaction_train, R_shape=R_full.shape).tocsr()
train_mask = R_train.toarray() > 0
test_mask = from_interactions_to_coo(interaction_test, R_shape=R_full.shape).toarray() > 0

/nmnt/media/home/boris/env3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [84]:
problem = IMCProblem(QAObjectiveLoss, X_comb, Y_comb, R_train, n_threads=8)

W_0 = random_state.normal(size=(X_comb.shape[1], K))
H_0 = random_state.normal(size=(Y_comb.shape[1], K))

W, H = W_0.copy(), H_0.copy()

W, H = imc_descent(problem, W, H,
                   step_fn,                  # the inner optimization
                   step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                   n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                   n_init_iterations=0,
                   return_history=True,      # Record the evolution of the matrices (W, H)
                   rtol=1e-5,                # relative stopping tolerance for the outer iterations
                   atol=1e-7,                # absolute tolerance
                   verbose=True,             # show the progress bar
                   check_product=True,       # use the product W H' for stopping
                   )

0it [00:00, ?it/s]
100%|██████████| 100/100 [00:06<00:00, 14.32it/s]


In [85]:
R_hat = get_prediction(X_comb, W, H, Y_comb)

print(relative_loss(R_full.toarray(), R_hat, mask=test_mask))
print(relative_loss(R_full.toarray(), R_hat, mask=train_mask))

0.2561068724565415
0.08070438832332932
